In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON, SPARQLExceptions
import requests
import re
from rdflib import Graph
import rdflib
# from rdfpandas.graph import to_dataframe
import pandas as pd
from tqdm import tqdm
import time
from mpire import WorkerPool
from mpire.utils import make_single_arguments
import pickle

In [78]:
df = pd.read_excel('./2008_2019_한국현대문학_참고문헌.xlsx')
df

,refebibl-id,artiId,type-code,type-name,title,author,pubilisher,pubi-year,page,arti-id,...,serno,doi,degree,university,department,url,citation-day,site-name,registration-day,conference-name
0,REF053056768,ART002536712,3,단행본,고등학교 문학(하),김윤식,㈜도서출판 디딤돌,2002,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,REF053056769,ART002536712,3,단행본,고등학교 문학,방민호,미래엔,2019,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,REF053056770,ART002536712,3,단행본,고등학교 문학,박호영,형설출판사,2003,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,REF053056771,ART002536712,3,단행본,고등학교 문학,조정래,해냄에듀,2014,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,REF053056772,ART002536712,3,단행본,국어문학(고급중학교2),최학,교육도서출판사,2014,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318923,REF016569235,ART001067385,1,학술지(정기간행물),5.18 그 삶과 죽음의 기록,NaN,풀빛,1996,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
318924,REF016569236,ART001067385,1,학술지(정기간행물),문학의 묵시록 이후,NaN,현대문학,2006.8,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
318925,REF016569237,ART001067385,1,학술지(정기간행물),오월은 푸르구나 우리들은 자란다 사람과 사람 사이의 신호,NaN,NaN,1986,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
318926,REF016569238,ART001067385,1,학술지(정기간행물),의미를 체현하는 육체,NaN,인간사랑,2003,17-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
df['type-name'].value_counts()

type-name
단행본           148836
학술지(정기간행물)    132594
기타자료           20264
학위논문           13213
인터넷자원           1775
학술대회논문          1678
보고서              568
Name: count, dtype: int64

In [64]:
# title_author_dict = df.loc[df['type-code']==3,['title','author']].drop_duplicates().to_dict('records')
title_author_dict = df.loc[:,['title','author']].dropna().drop_duplicates().to_dict('records')
len(title_author_dict)

178917

### LOD SPARQL

In [ ]:
# g = rdflib.Graph()
# g.parse('./book_ttl_20230320.ttl', format = 'ttl')
# # g.query()

In [51]:
sparql = SPARQLWrapper(
    "http://lod.nl.go.kr/sparql/"
)
sparql.setReturnFormat(JSON) # 리턴 형태 JSON으로

In [55]:
# 쿼리
# sparql.setQuery("""
# prefix dcterms: <http://purl.org/dc/terms/> 
# prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
# prefix nlon: <http://lod.nl.go.kr/ontology/>
# select ?s ?label where { 
#   ?s rdf:type <http://purl.org/ontology/bibo/Book> .   
#   ?s dcterms:title '문화의 위치' . 
#   ?s dc:creator ''}
#   limit 1
#     """
# )
# ret = sparql.queryAndConvert()
# ret             

{'head': {'vars': ['s', 'label']}, 'results': {'bindings': []}}

In [70]:
def get_lod(d):
    title =  d['title']
    author = d['author']
    query = (f"prefix dcterms: <http://purl.org/dc/terms/>\n"
    f"prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n"
    f"prefix nlon: <http://lod.nl.go.kr/ontology/>\n"
    f"select ?s ?label where {{\n"
    f"?s rdf:type <http://purl.org/ontology/bibo/Book> .\n"   
    f"?s dcterms:title '{title}' .\n"
    f"?s dc:creator '{author}'}}\n"
    f"limit 1")
    # print(query)
    sparql = SPARQLWrapper(
    "http://lod.nl.go.kr/sparql/"
    )
    sparql.setReturnFormat(JSON) # 리턴 형태 JSON으로
    try:
        sparql.setQuery(query)
        time.sleep(1.5)
        res = sparql.queryAndConvert()
        if len(res['results']['bindings'])!=0:
            res_dict = {}
            # res_dict['refebibl-id'] = l['refebibl-id']
            res_dict['title'] = title
            res_dict['author'] = author
            res_dict['uri'] = res['results']['bindings'][0]['s']['value']
            return res_dict
        else:
            return None
    except SPARQLExceptions.QueryBadFormed:
        # print('error')
        pass

In [67]:
title_author_dict_ = make_single_arguments(title_author_dict, generator=False)
len(title_author_dict_)

178917

In [ ]:
# lod_reulsts = []

# for l in tqdm(title_author_dict[:10]):
    # query = (f"prefix dcterms: <http://purl.org/dc/terms/>\n"
    # f"prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n"
    # f"prefix nlon: <http://lod.nl.go.kr/ontology/>\n"
    # f"select ?s ?label where {{\n"
    # f"?s rdf:type <http://purl.org/ontology/bibo/Book> .\n"   
    # f"?s dcterms:title '{l['title']}' .\n"
    # f"?s dc:creator '{l['author']}'}}\n"
    # f"limit 1")
    # sparql.setQuery(query)
    # time.sleep(0.5)
    # res = sparql.queryAndConvert()
    # if len(res['results']['bindings'])!=0:
    #     res_dict = {}
    #     # res_dict['refebibl-id'] = l['refebibl-id']
    #     res_dict['title'] = l['title']
    #     res_dict['author'] = l['author']
    #     res_dict['uri'] = res['results']['bindings'][0]['s']['value']
    #     lod_reulsts.append(res_dict)

In [71]:
with WorkerPool(n_jobs=10) as pool:
    lod_results = pool.map(get_lod,title_author_dict_,progress_bar=True)

100%|██████████| 178917/178917 [8:45:39<00:00,  1.53s/it]    


In [72]:
len(lod_results)

178917

In [73]:
# None 삭제
lod_results = [r for r in lod_results if r!=None]
len(lod_results)

13433

In [74]:
with open('./230706_한국현대문학_단행본_lod.pkl','wb') as f:
    pickle.dump(lod_results,f)
# with open('./230706_한국현대문학_단행본_lod.pkl','rb') as f:
#     lod_results = pickle.load(f)

In [79]:
lod_df = pd.DataFrame(lod_results)

In [80]:
# left join
df = df.merge(lod_df,how='left')
df

,refebibl-id,artiId,type-code,type-name,title,author,pubilisher,pubi-year,page,arti-id,...,doi,degree,university,department,url,citation-day,site-name,registration-day,conference-name,uri
0,REF053056768,ART002536712,3,단행본,고등학교 문학(하),김윤식,㈜도서출판 디딤돌,2002,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,REF053056769,ART002536712,3,단행본,고등학교 문학,방민호,미래엔,2019,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,REF053056770,ART002536712,3,단행본,고등학교 문학,박호영,형설출판사,2003,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,REF053056771,ART002536712,3,단행본,고등학교 문학,조정래,해냄에듀,2014,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,REF053056772,ART002536712,3,단행본,국어문학(고급중학교2),최학,교육도서출판사,2014,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318923,REF016569235,ART001067385,1,학술지(정기간행물),5.18 그 삶과 죽음의 기록,NaN,풀빛,1996,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
318924,REF016569236,ART001067385,1,학술지(정기간행물),문학의 묵시록 이후,NaN,현대문학,2006.8,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
318925,REF016569237,ART001067385,1,학술지(정기간행물),오월은 푸르구나 우리들은 자란다 사람과 사람 사이의 신호,NaN,NaN,1986,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
318926,REF016569238,ART001067385,1,학술지(정기간행물),의미를 체현하는 육체,NaN,인간사랑,2003,17-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
df[~pd.isna(df['uri'])]

,refebibl-id,artiId,type-code,type-name,title,author,pubilisher,pubi-year,page,arti-id,...,doi,degree,university,department,url,citation-day,site-name,registration-day,conference-name,uri
6,REF053056774,ART002536712,3,단행본,고등학교 교육과정 해설,교육과학기술부,한국보훈복지공단 신생인쇄조합,2008,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://lod.nl.go.kr/resource/KMO200930198
22,REF053056790,ART002536712,3,단행본,고등학교 문학교육의 형성과 흐름,최창헌,역락,2017,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://lod.nl.go.kr/resource/KMO201802560
31,REF053056542,ART002536697,3,단행본,주체문학론,김정일,조선로동당출판사,1992,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://lod.nl.go.kr/resource/KMO200026889
69,REF053056416,ART002536682,3,단행본,오만과 몽상,박완서,세계사,2002,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://lod.nl.go.kr/resource/KMO000075502
71,REF053056418,ART002536682,3,단행본,그 산이 정말 거기 있었을까,박완서,웅진출판사,1995,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://lod.nl.go.kr/resource/KMO199602213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318803,REF034542322,ART001871898,7,기타자료,구름을 잡으려고,주요섭,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://lod.nl.go.kr/resource/KMO000029644
318804,REF034542323,ART001871898,3,단행본,바닷가 소년,한남규,창작과비평,1992,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://lod.nl.go.kr/resource/KMO200624827
318808,REF034542327,ART001871898,3,단행본,인천시 다문화 분포의 공간적 특성에 관한 연구,신성희,인천발전연구원,2009,-,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://lod.nl.go.kr/resource/KMO200942321
318812,REF034542331,ART001871898,3,단행본,문학으로 인천을 읽다,이희환,작가들,2010,103-120,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://lod.nl.go.kr/resource/KMO201024154


In [85]:
# url = ret['results']['bindings'][0]['s']['value']
uri = df['uri'][69]
# url = re.sub('resource','data',url)
id = uri.split('/')[-1]

In [83]:
json_object = requests.get(f'http://lod.nl.go.kr/data/{id}?output=json').json()
json_object

{'http://lod.nl.go.kr/resource/KMO000075502': {'http://bibframe.org/vocab/extent': [{'type': 'literal',
    'value': '396p'},
   {'type': 'literal', 'value': '23cm'}],
  'http://lod.nl.go.kr/ontology/publicationPlace': [{'type': 'literal',
    'value': '서울'}],
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': [{'type': 'uri',
    'value': 'http://lod.nl.go.kr/ontology/Book'},
   {'type': 'uri', 'value': 'http://purl.org/ontology/bibo/Book'},
   {'type': 'uri', 'value': 'http://lod.nl.go.kr/ontology/OfflineMaterial'}],
  'http://purl.org/dc/elements/1.1/publisher': [{'type': 'literal',
    'value': '고려원'}],
  'http://lod.nl.go.kr/ontology/itemNumberOfNLK': [{'type': 'literal',
    'value': '박513ㅇㅁ'}],
  'http://lod.nl.go.kr/ontology/editionOfDDC': [{'type': 'literal',
    'value': '19'}],
  'http://lod.nl.go.kr/ontology/issuedYear': [{'type': 'literal',
    'value': '1985'}],
  'http://purl.org/dc/elements/1.1/creator': [{'type': 'literal',
    'value': '박완서'}],
  'http://lod.nl.go.k

In [84]:
json_object.keys()

dict_keys(['http://lod.nl.go.kr/resource/KMO000075502'])

In [41]:
json_object[uri].keys()

dict_keys(['http://bibframe.org/vocab/extent', 'http://lod.nl.go.kr/ontology/publicationPlace', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'http://purl.org/dc/elements/1.1/publisher', 'http://lod.nl.go.kr/ontology/itemNumberOfNLK', 'http://lod.nl.go.kr/ontology/editionOfDDC', 'http://lod.nl.go.kr/ontology/issuedYear', 'http://purl.org/dc/elements/1.1/creator', 'http://lod.nl.go.kr/ontology/editionOfKDC', 'http://purl.org/dc/elements/1.1/subject', 'http://lod.nl.go.kr/ontology/volumeOfNLK', 'http://lod.nl.go.kr/ontology/datePublished', 'http://purl.org/dc/terms/issued', 'http://lod.nl.go.kr/ontology/localHolding', 'http://lod.nl.go.kr/ontology/classificationNumberOfNLK', 'http://purl.org/dc/terms/creator', 'http://www.w3.org/2000/01/rdf-schema#label', 'http://lod.nl.go.kr/ontology/kdc', 'http://lod.nl.go.kr/ontology/typeOfData', 'http://lod.nl.go.kr/ontology/ddc', 'http://purl.org/dc/terms/title'])

In [42]:
json_object[uri]['http://purl.org/dc/elements/1.1/creator']

[{'type': 'literal', 'value': '박완서'}]

In [43]:
json_object[uri]['http://purl.org/dc/terms/creator']

[{'type': 'uri', 'value': 'http://lod.nl.go.kr/resource/KAC201007995'}]

In [44]:
json_object[uri]['http://purl.org/dc/terms/description']

KeyError: 'http://purl.org/dc/terms/description'

In [86]:
lod = requests.get(uri+'?output=ttf').text

In [87]:
lod

'@prefix schema: <http://schema.org/> .\n@prefix nlloc: <http://lod.nl.go.kr/location/> .\n@prefix geonames: <http://www.geonames.org/ontology#> .\n@prefix nlcon: <http://lod.nl.go.kr/countries/> .\n@prefix owl:   <http://www.w3.org/2002/07/owl#> .\n@prefix nllib: <http://lod.nl.go.kr/library/> .\n@prefix xsd:   <http://www.w3.org/2001/XMLSchema#> .\n@prefix skos:  <http://www.w3.org/2004/02/skos/core#> .\n@prefix rdfs:  <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix units: <http://lod.nl.go.kr/units/> .\n@prefix nlfloc: <http://lod.nl.go.kr/foreignLocation/> .\n@prefix geo:   <http://www.w3.org/2003/01/geo/wgs84_pos#> .\n@prefix nlgov: <http://lod.nl.go.kr/government/> .\n@prefix dcterms: <http://purl.org/dc/terms/> .\n@prefix vann:  <http://purl.org/vocab/vann/> .\n@prefix event: <http://purl.org/NET/c4dm/event.owl#> .\n@prefix foaf:  <http://xmlns.com/foaf/0.1/> .\n@prefix cc:    <http://creativecommons.org/ns#> .\n@prefix nla:   <http://lod.nl.go.kr/author/> .\n@prefix org:   <

In [88]:
g = Graph()

In [89]:
g.parse('http://lod.nl.go.kr/resource/KMO200303190',format='ttl')

<Graph identifier=Ncb56db3e96884e4fbd8ce223b268a60b (<class 'rdflib.graph.Graph'>)>

In [90]:
import pprint
for stmt in g:
    pprint.pprint(stmt)

(rdflib.term.URIRef('http://lod.nl.go.kr/resource/KMO200303190'),
 rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
 rdflib.term.Literal('문화의 위치 / 호미 바바 저; 나병철 옮김'))
(rdflib.term.URIRef('http://lod.nl.go.kr/resource/KMO200303190'),
 rdflib.term.URIRef('http://purl.org/dc/terms/hasFormat'),
 rdflib.term.URIRef('http://lod.nl.go.kr/resource/CNTS-00075130915'))
(rdflib.term.URIRef('http://lod.nl.go.kr/resource/KMO200303190'),
 rdflib.term.URIRef('http://purl.org/dc/elements/1.1/subject'),
 rdflib.term.Literal('위치'))
(rdflib.term.URIRef('http://lod.nl.go.kr/resource/KMO200303190'),
 rdflib.term.URIRef('http://purl.org/dc/elements/1.1/publisher'),
 rdflib.term.Literal('소명출판'))
(rdflib.term.URIRef('http://lod.nl.go.kr/resource/KMO200303190'),
 rdflib.term.URIRef('http://lod.nl.go.kr/ontology/ddcn'),
 rdflib.term.URIRef('http://lod.nl.go.kr/ddc/801_e21'))
(rdflib.term.URIRef('http://lod.nl.go.kr/resource/KMO200303190'),
 rdflib.term.URIRef('http://lod.nl.go.kr/ontology/ddc'),

In [91]:
for s, p ,o in g:
    print((s,p,o))

(rdflib.term.URIRef('http://lod.nl.go.kr/resource/KMO200303190'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.Literal('문화의 위치 / 호미 바바 저; 나병철 옮김'))
(rdflib.term.URIRef('http://lod.nl.go.kr/resource/KMO200303190'), rdflib.term.URIRef('http://purl.org/dc/terms/hasFormat'), rdflib.term.URIRef('http://lod.nl.go.kr/resource/CNTS-00075130915'))
(rdflib.term.URIRef('http://lod.nl.go.kr/resource/KMO200303190'), rdflib.term.URIRef('http://purl.org/dc/elements/1.1/subject'), rdflib.term.Literal('위치'))
(rdflib.term.URIRef('http://lod.nl.go.kr/resource/KMO200303190'), rdflib.term.URIRef('http://purl.org/dc/elements/1.1/publisher'), rdflib.term.Literal('소명출판'))
(rdflib.term.URIRef('http://lod.nl.go.kr/resource/KMO200303190'), rdflib.term.URIRef('http://lod.nl.go.kr/ontology/ddcn'), rdflib.term.URIRef('http://lod.nl.go.kr/ddc/801_e21'))
(rdflib.term.URIRef('http://lod.nl.go.kr/resource/KMO200303190'), rdflib.term.URIRef('http://lod.nl.go.kr/ontology/ddc'), rdflib.ter

In [65]:
creator = [p for p in g.predicates()][9]

In [74]:
creator

rdflib.term.URIRef('http://purl.org/dc/elements/1.1/creator')

In [48]:
to_dataframe(g)

,bibframe:extent{Literal}[0],bibframe:extent{Literal}[1],nlon:bibliography{Literal},nlon:classificationNumberOfNLK{Literal},nlon:datePublished{Literal}(xsd:dateTime),nlon:ddc{Literal},nlon:ddcn{URIRef},nlon:editionOfDDC{Literal},nlon:editionOfKDC{Literal},nlon:issuedYear{Literal},...,dcterms:language{URIRef},dcterms:title{Literal},bibo:identifier{Literal},bibo:isbn{Literal},rdf:type{URIRef}[0],rdf:type{URIRef}[1],rdf:type{URIRef}[2],rdfs:label{Literal},rdfs:seeAlso{URIRef},owl:sameAs{URIRef}
nlk:KMO200303190,23cm,488p,색인수록,809.07,2023-06-05T21:35:27,801.95,nlddc:801_e21,21,4,2002,...,http://id.loc.gov/vocabulary/languages/kor,문화의 위치,UB20030003479,8956260125,nlon:Book,nlon:OfflineMaterial,bibo:Book,문화의 위치 / 호미 바바 저; 나병철 옮김,http://book.naver.com/bookdb/book_detail.php?b...,http://data.nanet.go.kr/ontology/NANET_BIB_MON...
